In [3]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import numpy as np
import pandas as pd
import time
import re
from tqdm import tqdm

In [2]:
seed_address_list=[]
seed_page="https://www.immoregion.fr/vente/maison?page="

for i in range(1,50,1):
    current_page=seed_page+str(i)
    page=requests.get(current_page)
    page_content=BeautifulSoup(page.content,"html.parser")
    for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')}):
        if "id-" in link["href"]:
            seed_address_list.append("https://www.immoregion.fr"+link["href"])
    print("{} round".format(i))
seed_address_list=list(np.unique(seed_address_list))

1 round
2 round
3 round
4 round
5 round
6 round
7 round
8 round
9 round
10 round
11 round
12 round
13 round
14 round
15 round
16 round
17 round
18 round
19 round
20 round
21 round
22 round
23 round
24 round
25 round
26 round
27 round
28 round
29 round
30 round
31 round
32 round
33 round
34 round
35 round
36 round
37 round
38 round
39 round
40 round
41 round
42 round
43 round
44 round
45 round
46 round
47 round
48 round
49 round


In [4]:
len(seed_address_list)

978

In [30]:
json_list=[]
for link in tqdm(seed_address_list):
    try:
        page=requests.get(link)
        if page.status_code==200:
            page_content=BeautifulSoup(page.content,"html.parser")
            
            price=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[0].find_all("div")[0].text
            area=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[3].find_all("div")[0].text
            furnished=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[2].find_all("div")[0].text
            integral_area=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[4].find_all("div")[0].text
            nmbr_of_rooms=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[5].find_all("div")[0].text
            ref=page_content.find_all("h1",class_="KeyInfoBlockStyle__PdpTitle-sc-1o1h56e-2 hWEtva")[0].find_all("span")[0].text 
            description=page_content.find_all("div",class_="DescriptionStyle__DescriptionContentDropdown-dqxmgt-3 dyQEIL")[0].find_all("div")[0].text
            link_address=link
            scrapping_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            agency=page_content.find_all("h3",class_="FormAgencyInfoStyle__AgencyName-sc-10hhmvq-5 jfZIgK")[0].text
            address=page_content.find_all("div",class_="block-localisation-address")[0].text

            json_data={"Ref_no":ref,"Price":price,"Surface":area,"furnished":furnished,"integral_area":integral_area,"No_room":nmbr_of_rooms,"Description":description,"Link":link_address,"scrapping_time":scrapping_time,"agency":agency,"address":address}
            json_list.append(json_data)
            time.sleep(2)
        else:
            pass
    except:
        pass

100%|██████████| 978/978 [45:42<00:00,  2.80s/it]


In [34]:
pd.DataFrame(json_list).iloc[0].loc["Link"]

'https://www.immoregion.fr/vente/maison/aboncourt/id-6772175.html'

In [36]:
link

'https://www.immoregion.fr/vente/maison/zoufftgen/id-6738619.html'

In [37]:
page=requests.get(link)
if page.status_code==200:
    page_content=BeautifulSoup(page.content,"html.parser")

In [204]:
for  container in page_content.find_all("div",class_="feature-bloc"):
    print(container.find_all("div",class_="feature-bloc-content-title-name")[0].text)
    if "Détails de la vente" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
        for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
            if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Prix de vente":
                price=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Prix hors honoraires":
                price_without_honor=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Meublé":
                furnished=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text

    elif "Général" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
        for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
            if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Surface":
                area=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Terrain":
                integral_area=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Nombre de pièces":
                nmbr_of_rooms=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Nombre de chambres":
                nmbr_of_bedrooms=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Année de construction":
                constr_year=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text

    elif "Intérieur" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
        for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
            if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Salon":
                salon_area=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Salles de bain":
                nmbr_of_bathrooms=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Toilettes séparées":
                nmbr_of_sep_toilettes=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text

    elif "Extérieur" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
        for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
            if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Terrasse":
                terrasse=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Place(s) de parking en extérieur":
                parkinglots=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text

    elif "Autres" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
        for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
            if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Cave":
                cave=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Piscine":
                pool=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text


[2 for var is None else var in [ref,price,area,integral_area,furnished,salon_area,nmbr_of_bathrooms,nmbr_of_bedrooms,nmbr_of_rooms,nmbr_of_sep_toilettes,constr_year,description,link_address,scrapping_time,agency,address,terrasse,parkinglots,cave,pool]]

json_data={"Ref_no":ref,"Price":price,"Surface":area,"integral_area":integral_area,"furnished":furnished,"salon_area":salon_area,"No_room":nmbr_of_rooms,"No_bedrooms":nmbr_of_bedrooms,"construction_year":constr_year,"Description":description,"Link":link_address,"scrapping_time":scrapping_time,"agency":agency,"address":address,"no_bathrooms":nmbr_of_bathrooms,"no_sep_toilettes":nmbr_of_sep_toilettes,"terrasse":terrasse,"parking":parkinglots,"cave":cave,"pool":pool}

SyntaxError: invalid syntax (<ipython-input-204-61e8117ffbc7>, line 49)

In [198]:
json_data

{'Ref_no': 'Réf 6738619',
 'Price': '740\xa0000 €',
 'Surface': '183 m²',
 'integral_area': '7,71 ares',
 'furnished': 'NC',
 'salon_area': '50 m²',
 'No_room': '7',
 'No_bedrooms': '4',
 'construction_year': '2011',
 'Description': "Splendide maison d'architecte F7, construite en 2011 et implantée sur un terrain de 7,71 ares avec piscine chauffée.\nElle se compose en rez-de-chaussée, d'un hall d'entrée toute hauteur et traversant, d'un vaste salon|séjour de 50m2, d'une cuisine ouverte toute équipée attenant à un cellier pouvant communiquer directement avec l'extérieur ou avec le double garage ainsi que d'un bureau et d'un espace vestiaire avec placards intégré fait sur mesure avec wc séparé.\nA l'étage une mezzanine spectaculaire avec plancher en verre donnant vu sur le jardin et séparant le coin nuit enfant composé de deux chambres à coucher et d'une salle de bains de l'espace parentale de 26m2 comprenant une seconde salle de bains privative, d'un dressing toute hauteur et d'une cham

In [35]:
clear

In [369]:
#get new links which are similar to the one you have looked at

seed_address="https://www.immoregion.fr/vente/maison/metz/id-6708871.html"
seed_address_list=[]

for i in range(0,3,1):
    if len(seed_address_list)==0:
        #add original seed_address
        print("seed_address_list is empty")
        seed_address_list.append(seed_address) 
        #read the original seed address and if reading process successful load html in var pagecontent
        page=requests.get(seed_address)                 
        if page.status_code==200:
            print("seed page loaded")
            page_content=BeautifulSoup(page.content,"html.parser")
            #iterate over 10 found links to other offers
            for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-3:]:
                new_link="https://www.immoregion.fr"+link["href"]
                seed_address_list.append(new_link)
                print("link from seed_page added")
                print(new_link)
            counter=1
        else:
            counter=1
        print("erste Schleife zuende")
        print(seed_address_list)
        print("\n")    
    else:
        print("seed_address not empty")
        print("counter: {}".format(counter))
        for link in seed_address_list[counter:]:
            page=requests.get(link)
            if page.status_code==200:
                page_content=BeautifulSoup(page.content,"html.parser")
                #iterate over n found links to other offers
                print("Länge Link Liste: {}".format(len(page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')}))))
                for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-3:]:
                    new_link="https://www.immoregion.fr"+link["href"]
                    new_links.append(new_link)
                    print("link from seed_page added")
                    print(new_link)

            else:
                pass
        print("\n")
        [seed_address_list.append(address) for address in new_links]
        print("seed_address_list updated")
        print("Länge Seed address_list: {}".format(len(seed_address_list)))
        counter+=3
        print("Counter updated: {}".format(counter))
        new_links=[]
        print("\n")